In [1]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

In [2]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

In [25]:
def eye_aspect_ratio(eye,show=True):
    A = distance.euclidean(eye[1],eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A+B)/(2.0 * C)
    if show == True :
        print("EAR :",ear)
    return ear # WE GOT THE EAR VALUE NOW

def mouth_aspect_ratio(mouth,show=True):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    if show == True:
        print("mar :",(A/C))
    return A/C  #// we got mar value now

def mouth_over_eye(eye,show=True):
    ear = eye_aspect_ratio(eye,False)
    mar = mouth_aspect_ratio(eye,False)
    mouth_eye = mar/ear
    if show == True:
        print("mouth_eye :",mouth_eye)
    return mouth_eye

def circularity(eye): # PUC 
    # puc = (4 pi area)/(perimeter sq)
    
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0 # perimeter
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    puc = (4 * math.pi * Area) /(p**2)
    print("puc :",puc)
    
    return (puc)



In [4]:
def getFrame(sec):
    start = 180000 # the 3 minute mark
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000) # gets one frame ,frame at  3 minte mark+ sec
    hasFrames,image = vidcap.read()
    print("got a frame at",start + sec*1000,"sec mark , status :",hasFrames)
    return hasFrames, image

In [14]:
from scipy.spatial import distance
import cv2
data=[]
labels = []
face_landmarks=[]
for j in [60]:
    for i in [10]:
        loc='/home/v/Desktop/Drowsiness detection/Fold5_part2'+ '/'+ str(j) +'/' + str(i) + '.mov'
        print("file path :",loc)
        vidcap = cv2.VideoCapture(loc)
        print("file_loaded")
        sec = 0
        frameRate = 1
        success,image = getFrame(sec)
        count = 0
#         print("got a frame at",sec,"sec mark , status :",success)
        
        while success and count < 10 :#240
            print("helo world")
            landmarks = extract_face_landmarks(image)
            face_landmarks.append(landmarks)
            try :
                if sum(sum(landmarks)) != 0 and type(sum(sum(landmarks))) != type(None) :
                    count += 1
                    data.append(landmarks)
                    sec = sec + frameRate
                    sec = round(sec,2)
                    success,image = getFrame(sec)
#                     print("got a frame at",sec,"sec mark , status :",success)
                    print("count :",count)
                else :
                    sec = sec + frameRate
                    sec = round(sec,2)
                    success, image = getFrame(sec)
                    print("not detected")
            except :
                print("failed in getting a land mark")
                print("type",type(sum(sum(landmarks))))
                print("sum",sum(sum(landmarks)))
                sec = sec + frameRate
                sec = round(sec,2)
                count +=1
                print("sec :",sec,"count:",count)
                

file path : /home/v/Desktop/Drowsiness detection/Fold5_part2/60/10.mov
file_loaded
helo world
count : 1
helo world
count : 2
helo world
count : 3
helo world
count : 4
helo world
count : 5
helo world
count : 6
helo world
count : 7
helo world
count : 8
helo world
count : 9
helo world
count : 10


In [15]:
import numpy as np
data = np.array(data)
labels = np.array(labels)

In [30]:
features = []
for d in data:
  eye = d[36:68]
  ear = eye_aspect_ratio(eye)
  mar = mouth_aspect_ratio(eye)
  cir = circularity(eye)
  mouth_eye = mouth_over_eye(eye)
  features.append([ear, mar, cir, mouth_eye])
    

IndexError: index 1 is out of bounds for axis 0 with size 0

In [29]:
features
features = np.array(features)
features.shape
data.shape

(10, 1, 68, 2)